In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [2]:
airline_file = "Data/airlines.csv"
airline_df = pd.read_csv(airline_file)
airline_df.head()

,Airline ID,Name,Alias,IATA,ICAO,Callsign,Country,Active
0,-1,Unknown,\N,-,NaN,\N,\N,Y
1,1,Private flight,\N,-,NaN,NaN,NaN,Y
2,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N


In [3]:
airline_clean_df = airline_df[["Airline ID", "Name", "ICAO", "Country", "Active"]]
airline_clean_df = airline_clean_df.rename(columns = {"Name": "Operator"})
airline_clean_df

,Airline ID,Operator,ICAO,Country,Active
0,-1,Unknown,NaN,\N,Y
1,1,Private flight,NaN,NaN,Y
2,2,135 Airways,GNL,United States,N
3,3,1Time Airline,RNX,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,WYT,United Kingdom,N
...,...,...,...,...,...
6157,21248,GX Airlines,CBG,China,Y
6158,21251,Lynx Aviation (L3/SSX),SSX,United States,N
6159,21268,Jetgo Australia,\N,Australia,Y
6160,21270,Air Carnival,\N,India,Y


In [4]:
# Changing the name of Aeroflot Russian Airlines to Aeroflot
airline_clean_df = airline_clean_df.set_index('Operator')
airline_clean_df = airline_clean_df.rename(index={'Aeroflot Russian Airlines': 'Aeroflot',
                                      'Avianca - Aerovias Nacionales de Colombia': 'Avianca'})

airline_clean_df

,Airline ID,ICAO,Country,Active
Operator,,,,
Unknown,-1,NaN,\N,Y
Private flight,1,NaN,NaN,Y
135 Airways,2,GNL,United States,N
1Time Airline,3,RNX,South Africa,Y
2 Sqn No 1 Elementary Flying Training School,4,WYT,United Kingdom,N
...,...,...,...,...
GX Airlines,21248,CBG,China,Y
Lynx Aviation (L3/SSX),21251,SSX,United States,N
Jetgo Australia,21268,\N,Australia,Y


In [5]:
airline_clean_df.reset_index()

,Operator,Airline ID,ICAO,Country,Active
0,Unknown,-1,NaN,\N,Y
1,Private flight,1,NaN,NaN,Y
2,135 Airways,2,GNL,United States,N
3,1Time Airline,3,RNX,South Africa,Y
4,2 Sqn No 1 Elementary Flying Training School,4,WYT,United Kingdom,N
...,...,...,...,...,...
6157,GX Airlines,21248,CBG,China,Y
6158,Lynx Aviation (L3/SSX),21251,SSX,United States,N
6159,Jetgo Australia,21268,\N,Australia,Y
6160,Air Carnival,21270,\N,India,Y


In [6]:
# airline_clean_df = airline_clean_df['Country'].replace({'ALASKA': 'United States',
#                                                         'AVIANCA': 'Colombia',
#                                                         'Russia]]': 'Russia'
#                                                         })

In [7]:
crash_file = "Data/Airplane_Crashes_and_Fatalities_Since_1908.csv"
crash_df = pd.read_csv(crash_file)
crash_df.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,7/12/1912,6:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,8/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,9/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [8]:
crash_clean_df = crash_df[["Date", "Location", "Operator", "Type", "Aboard", "Fatalities", "Summary"]].copy()
crash_clean_df["Date"] = pd.to_datetime(crash_clean_df["Date"])
crash_clean_df = crash_clean_df[crash_clean_df["Date"].dt.year >= 1970]

crash_clean_df = crash_clean_df.reset_index()

In [9]:
crash_clean_df[crash_clean_df['Operator'].str.contains('/', na=False)]

,index,Date,Location,Operator,Type,Aboard,Fatalities,Summary
80,2436,1970-11-24,"Nar Can Tho, South Vietnam",Military - Vietnamese Air Force / Military - U...,de Havilland RU-6A Beaver /Bell UH-1H,18.0,18.0,"Mid-air collision at 1,200 ft."
93,2449,1971-01-09,"Edison, New Jersey",American Airlines / Private,Boeing B-707-323 / Cessna 150,21.0,2.0,"Midair collision at 3,000 ft. Failed to see a..."
119,2475,1971-06-06,"Near Duarte, California",Hughes Airwest / Military - US Marine Air Corps,McDonnell Douglas DC-9-31 / F4-B,51.0,50.0,The DC-9 was climbing to FL 330 under radar co...
129,2485,1971-07-30,"Near Morioko, Japan",All Nippon Airways / Japanese Air Force,Boeing B-727-281 / Air Force F86F,164.0,163.0,A Japan Air Self Defense Force F-86F fighter c...
214,2570,1972-06-29,"Appleton, Wisconsin",North Central Airlines / Air Wisconsin,Convair CV-580/De Hav. Twin Otter 100,13.0,13.0,Both aircraft were proceeding under VFR gettin...
222,2578,1972-07-29,"Villavicencio, Colombia",AVIANCA / AVIANCA,Douglas DC-3 / Douglas DC-3,38.0,38.0,Midair collision. Seventeen aboard the HK-134 ...
260,2616,1972-12-20,"Chicago, Illinois",Delta Air Lines/ North Central Airlines,Convair CV-880 / McDonnell DC-9-31,133.0,10.0,"The Delta Convair, taxing across Runway 27L, a..."
286,2642,1973-03-05,"Near Nantes, France",Lineas Aereas de Espana/Spantax SA Transportes,MD Douglas DC-9-32/Convair CV-990A,182.0,68.0,Midair collision. Convair crew improperly asse...
292,2648,1973-04-12,"Sunnyvale, California",Military - U.S. Navy / NASA,Lockheed P-3C / Convair CV-990-30A-5,17.0,16.0,A Navy P-3C collided with a NASA research jet ...
321,2677,1973-09-04,"Gary, Indiana",Aegler Stuart Flying Service / Private,Piper PA-30 / Cessna 337A,5.0,4.0,"The Piper, landing on Runway 20 collilded with..."


In [10]:
# crash_clean_df['Operator', 'Type'] = crash_clean_df['Operator', 'Type'].str.split("/") 
# crash_clean_df.explode('Operator', 'Type').reset_index(drop=False)


In [11]:
crash_clean_df.head()

,index,Date,Location,Operator,Type,Aboard,Fatalities,Summary
0,1237,1970-02-13,"Pin-la-Garenne, France",Avia-Taxi,Cessna 402,10.0,10.0,NaN
1,1643,1973-05-11,"Near Semipalatinsk, USSR",Aeroflot,Ilyushin IL-18B,61.0,61.0,NaN
2,1807,1973-10-12,"Acapulco, Mexico",Avionas Banamex,de Havilland DH-125-1A,8.0,8.0,Crashed while approaching to land.
3,1965,1973-10-15,"Fort Smith AFB, Oklahoma",Military - U.S. Air Force,Lockheed C-130E Hercules,7.0,7.0,Crashed into a mountain.
4,2003,1975-07-31,"Santiago, Chili",Military - Chilian Air Force,Douglas C-47,10.0,10.0,NaN


In [12]:
crash_clean_df.iloc[80:100]

,index,Date,Location,Operator,Type,Aboard,Fatalities,Summary
80,2436,1970-11-24,"Nar Can Tho, South Vietnam",Military - Vietnamese Air Force / Military - U...,de Havilland RU-6A Beaver /Bell UH-1H,18.0,18.0,"Mid-air collision at 1,200 ft."
81,2437,1970-11-27,"Anchorage, Alaska",Capital Airlines,McDonnell Douglas DC-8-63CF,229.0,47.0,The military charter overran the runway during...
82,2438,1970-11-27,"Near Nha Trang, South Vietnam",Military - U.S. Air Force,Fairchild C-123K,79.0,79.0,Crashed into high ground on a mountainside whi...
83,2439,1970-11-29,"Near Cam Ranh, South Vietnam",Military - U.S. Air Force,Fairchild C-123K,44.0,42.0,Crashed into the jungle 15 miles from it desti...
84,2440,1970-12-02,"Dhaka, Bangladesh",Cargolux,Canadair CL-44J,4.0,4.0,The cargo plane crashed while attempting to la...
85,2441,1970-12-05,"Delhi, India",Jamair,Douglas DC-3,16.0,5.0,The No. 2 engine failed on takeoff due to fuel...
86,2442,1970-12-07,"Near Constanta, Romania",Tarom,BAC One-Eleven 424EU,27.0,18.0,Crashed 4 kms short of the runway on approach ...
87,2443,1970-12-19,"Panarah, Pakistan",Aeroflot,Antonov AN-22,25.0,25.0,Crashed during an emergency landing with engin...
88,2444,1970-12-28,"St. Thomas, Virgin Islands",Trans Caribbean Airways,Boeing B-727-2A7,55.0,2.0,The aircraft experienced a hard landing which ...
89,2445,1970-12-30,"Near Shemshernagar, Pakistan",Pakistan International Airlines,Fokker F-27 Friendship 200,35.0,7.0,Crashed and burned on landing.


In [13]:
crash_clean_df = crash_clean_df.replace({'United Air lines': 'United Airlines',
                                         'AVIANCA': 'Avianca',
                                         'Varig': ' Varig Log',
                                         'Compania Dominicana de Aviacion': 'Dominicana de Aviaci',
                                         'Aviogenex (Yugoslavia)': 'Aviogenex',
                                         'Stirling Airways': 'Stirling Airlines',
                                         'Japan Air Lines': 'Japan Airlines',
                                         'Cathay PacifiAirways': 'Cathay Pacific',
                                         'British European Airways': 'British Airways',
                                         'Spantax': 'Spantax S.A.',
                                         'Eastern Air Lines': 'Balitmore Airways',
                                         'Alia Royal Jordanian Airlines': ' Royal Jordanian',
                                         'Libya Arab Airlines': ' Libyan Airlines',
                                         'Turkish Airlines(THY)': 'Turkish Airlines',
                                         'Martinair Holland NV': 'Martinair',
                                         'Ceskoslevenske Aerolinie': 'Czech Airlines',
                                         'Inex Adria Aviopromet': 'Adria Airways',
                                         'TAP (Air Portugal)': 'TAP Portugal',
                                         'Air India': 'Air India Limited',
                                         'Loftledidir IcelandiAirlines': 'Icelandair',
                                         'Iran National Airlines': 'Iran Air',
                                         'Dan Air Services': 'Dan-Air London',
                                         'Korean Airlines': ' Korean Air',
                                         'Arrow Airways': 'Arrow Air',
                                         'Mexicana': 'Mexicana de Aviaci',
                                         'Thai Airways': 'Thai Airways International',
                                         'Tajikistan Airlines': 'Tajikistan International Airlines',
                                         'Cebu PacifiAir': ' Cebu Pacific',
                                         'China Airlines (Taiwan)': 'China Airlines',
                                         'EgyptAir': 'Egyptair',
                                         'Scandinavian Airlines (SAS)': 'Scandinavian Airlines System',
                                         'Flash Air': 'Flash Airlines',
                                         'Subir (S7)': 'S7 Airlines',
                                         'Pulkovo Airlines': 'Rossiya-Russian Airlines',
                                         'TAM (Brazil)': 'TAM Brazilian Airlines',
                                         'One-Two-Go Airlines': 'One Two Go Airlines'})

In [14]:
airline_crash_df = pd.merge(airline_clean_df, crash_clean_df, how='inner', on='Operator')
airline_crash_df

,Operator,Airline ID,ICAO,Country,Active,index,Date,Location,Type,Aboard,Fatalities,Summary
0,Aloha Airlines,22,AAH,United States,Y,3764,1988-04-28,"Maui, Hawaii",Boeing B-737-297,95.0,1.0,The aircraft suffered separation of the top of...
1,American Airlines,24,AAL,United States,Y,2895,1976-04-27,"St. Thomas, Virgin Islands",Boeing B-727-95,88.0,37.0,"After touching down 2,200 feet down the runway..."
2,American Airlines,24,AAL,United States,Y,3137,1979-05-25,"Chicago O'Hare, Illinois",McDonnell Douglas DC-10-10,271.0,271.0,"During takeoff just, as the plane lifted from ..."
3,American Airlines,24,AAL,United States,Y,3182,1979-11-15,"Near Chicago, Illinois",Boeing B-727,78.0,0.0,While en route from Chicago to Washington an e...
4,American Airlines,24,AAL,United States,Y,3549,1985-04-16,"Las Cruces, New Mexico",Boeing B-727-227,89.0,0.0,While cruising at FL 350 a loud noise was hear...
...,...,...,...,...,...,...,...,...,...,...,...,...
871,Spantax S.A.,18140,BXS,Spain,N,2592,1972-09-30,"Madrid, Spain",Douglas C-47B,6.0,1.0,"The student pilot rotated too quickly, the pla..."
872,Spantax S.A.,18140,BXS,Spain,N,2608,1972-12-03,"Tenerife, Canary Islands",Convair CV-990-30A-5 Coronado,155.0,155.0,Reached a height of 300 feet when the plane s...
873,Spantax S.A.,18140,BXS,Spain,N,3237,1980-08-13,"Sa Cabaneta, Spain",Learjet 35A,4.0,4.0,Slamed into a mountain while on approach.
874,Spantax S.A.,18140,BXS,Spain,N,3378,1982-09-13,"Near Malaga, Spain",McDonnell Douglas DC-10-30CF,394.0,50.0,The aircraft crashed during takeoff after inte...


In [15]:
airline_crash_df.drop(columns=['index'])

,Operator,Airline ID,ICAO,Country,Active,Date,Location,Type,Aboard,Fatalities,Summary
0,Aloha Airlines,22,AAH,United States,Y,1988-04-28,"Maui, Hawaii",Boeing B-737-297,95.0,1.0,The aircraft suffered separation of the top of...
1,American Airlines,24,AAL,United States,Y,1976-04-27,"St. Thomas, Virgin Islands",Boeing B-727-95,88.0,37.0,"After touching down 2,200 feet down the runway..."
2,American Airlines,24,AAL,United States,Y,1979-05-25,"Chicago O'Hare, Illinois",McDonnell Douglas DC-10-10,271.0,271.0,"During takeoff just, as the plane lifted from ..."
3,American Airlines,24,AAL,United States,Y,1979-11-15,"Near Chicago, Illinois",Boeing B-727,78.0,0.0,While en route from Chicago to Washington an e...
4,American Airlines,24,AAL,United States,Y,1985-04-16,"Las Cruces, New Mexico",Boeing B-727-227,89.0,0.0,While cruising at FL 350 a loud noise was hear...
...,...,...,...,...,...,...,...,...,...,...,...
871,Spantax S.A.,18140,BXS,Spain,N,1972-09-30,"Madrid, Spain",Douglas C-47B,6.0,1.0,"The student pilot rotated too quickly, the pla..."
872,Spantax S.A.,18140,BXS,Spain,N,1972-12-03,"Tenerife, Canary Islands",Convair CV-990-30A-5 Coronado,155.0,155.0,Reached a height of 300 feet when the plane s...
873,Spantax S.A.,18140,BXS,Spain,N,1980-08-13,"Sa Cabaneta, Spain",Learjet 35A,4.0,4.0,Slamed into a mountain while on approach.
874,Spantax S.A.,18140,BXS,Spain,N,1982-09-13,"Near Malaga, Spain",McDonnell Douglas DC-10-30CF,394.0,50.0,The aircraft crashed during takeoff after inte...


In [17]:
airline_crash_df[airline_crash_df['Operator'] == 'Thai Airways International']

,Operator,Airline ID,ICAO,Country,Active,index,Date,Location,Type,Aboard,Fatalities,Summary
779,Thai Airways International,4940,THA,Thailand,Y,3215,1980-04-27,"Don Muang, Thailand",Hawker Siddeley HS-748-2,53.0,44.0,"The aircraft entered a thunderstorm, was caugh..."
780,Thai Airways International,4940,THA,Thailand,Y,3548,1985-04-15,"Near Phuket, Thailand",Boeing B-737-200,11.0,11.0,The aircraft hit high ground during good weath...
781,Thai Airways International,4940,THA,Thailand,Y,3708,1987-08-31,"Off Ko Phuket, Thiland",Boeing B-737-2P5,83.0,83.0,The aircraft stalled and crashed into the ocea...
782,Thai Airways International,4940,THA,Thailand,Y,4118,1992-07-31,"Near Kathmandu, Nepal",Airbus A310-304,113.0,113.0,After experiencing difficulty in deploying the...
783,Thai Airways International,4940,THA,Thailand,Y,4606,1998-12-11,"Near Surat Thani, Thailand",Airbus A-310-204,146.0,102.0,The plane crashed 2 miles southwest of Surat T...
784,Thai Airways International,4940,THA,Thailand,Y,4773,2001-03-03,"Bangkok, Thailand",Boeing B-737-4D7,5.0,1.0,The aircraft exploded and caught fire 35 minut...
